In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp5oojonum".


In [ ]:
%%writefile matrix_gpu.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

__global__ void matrixMultiplyGPU(float *A, float *B, float *C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        float sum = 0.0f;
        for (int k = 0; k < N; k++) {
            sum += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}

int main(int argc, char **argv) {
    int M = (argc > 1) ? atoi(argv[1]) : 1024; // allow matrix size as input
    for (int i = 1; i <= M / 512; i *= 2) {
        int N = 512 * i;
        size_t size = N * N * sizeof(float);

        float *A;
        cudaMallocManaged(&A, size);
        float *B;
        cudaMallocManaged(&B, size);
        float *C;
        cudaMallocManaged(&C, size);

        for (int i = 0; i < N * N; i++) {
            A[i] = rand() % 100 / 100.0f;
            B[i] = rand() % 100 / 100.0f;
        }

        dim3 threadsPerBlock(16, 16);
        dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                           (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

        clock_t start = clock();
        matrixMultiplyGPU<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, N);
        cudaDeviceSynchronize();
        clock_t end = clock();

        double elapsed = (double)(end - start) / CLOCKS_PER_SEC;
        printf("GPU execution time (N=%d): %f seconds\n", N, elapsed);

        cudaFree(A); cudaFree(B); cudaFree(C);
    }
    return 0;
}

Writing matrix_gpu.cu


In [ ]:
!nvcc -arch=sm_75 matrix_gpu.cu -o matrix_gpu_run

In [ ]:
!./matrix_gpu_run 2048

GPU execution time (N=512): 0.002405 seconds
GPU execution time (N=1024): 0.012200 seconds
GPU execution time (N=2048): 0.088387 seconds


In [ ]:
%%writefile matrix_gpu_tiled.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define TILE_WIDTH 16

__global__ void matrixMultiplyTiled(float *A, float *B, float *C, int N) {
    __shared__ float ds_A[TILE_WIDTH][TILE_WIDTH];
    __shared__ float ds_B[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x; int by = blockIdx.y;
    int tx = threadIdx.x; int ty = threadIdx.y;

    int Row = by * TILE_WIDTH + ty;
    int Col = bx * TILE_WIDTH + tx;

    float Pvalue = 0.0;
    for (int m = 0; m < (N + TILE_WIDTH - 1) / TILE_WIDTH; ++m) {
        if (Row < N && (m*TILE_WIDTH+tx) < N)
            ds_A[ty][tx] = A[Row * N + m * TILE_WIDTH + tx];
        else
            ds_A[ty][tx] = 0.0f;

        if (Col < N && (m*TILE_WIDTH+ty) < N)
            ds_B[ty][tx] = B[(m*TILE_WIDTH + ty) * N + Col];
        else
            ds_B[ty][tx] = 0.0f;

        __syncthreads();

        for (int k = 0; k < TILE_WIDTH; ++k)
            Pvalue += ds_A[ty][k] * ds_B[k][tx];
        __syncthreads();
    }

    if (Row < N && Col < N)
        C[Row * N + Col] = Pvalue;
}

int main(int argc, char **argv) {
    int M = (argc > 1) ? atoi(argv[1]) : 1024; // allow matrix size as input
    for (int i = 1; i <= M / 512; i *= 2) {
        int N = 512 * i;
        size_t size = N * N * sizeof(float);

        float *A;
        cudaMallocManaged(&A, size);
        float *B;
        cudaMallocManaged(&B, size);
        float *C;
        cudaMallocManaged(&C, size);

        for (int i = 0; i < N * N; i++) {
            A[i] = rand() % 100 / 100.0f;
            B[i] = rand() % 100 / 100.0f;
        }

        dim3 threadsPerBlock(16, 16);
        dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                           (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

        clock_t start = clock();
        matrixMultiplyTiled<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, N);
        cudaDeviceSynchronize();
        clock_t end = clock();

        double elapsed = (double)(end - start) / CLOCKS_PER_SEC;
        printf("GPU execution time (N=%d): %f seconds\n", N, elapsed);

        cudaFree(A); cudaFree(B); cudaFree(C);
    }
    return 0;
}

Overwriting matrix_gpu_tiled.cu


In [ ]:
!nvcc -arch=sm_75 matrix_gpu_tiled.cu -o matrix_gpu_tiled_run

In [ ]:
!./matrix_gpu_tiled_run 2048

GPU execution time (N=512): 0.002101 seconds
GPU execution time (N=1024): 0.009680 seconds
GPU execution time (N=2048): 0.060095 seconds


In [ ]:
%%writefile matrix_gpu_cublas.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cublas_v2.h>

int main(int argc, char **argv) {
    int M = (argc > 1) ? atoi(argv[1]) : 1024; // allow matrix size as input
    cublasHandle_t handle;
    cublasCreate(&handle);

    const float alpha = 1.0f;
    const float beta = 0.0f;

    for (int i = 1; i <= M / 512; i *= 2) {
        int N = 512 * i;
        size_t size = N * N * sizeof(float);

        float *A;
        cudaMallocManaged(&A, size);
        float *B;
        cudaMallocManaged(&B, size);
        float *C;
        cudaMallocManaged(&C, size);

        for (int i = 0; i < N * N; i++) {
            A[i] = rand() % 100 / 100.0f;
            B[i] = rand() % 100 / 100.0f;
        }

        dim3 threadsPerBlock(16, 16);
        dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                           (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

        clock_t start = clock();
        cublasSgemm(handle,
              CUBLAS_OP_N,
              CUBLAS_OP_N,
              N, N, N,
              &alpha,
              B, N,
              A, N,
              &beta,
              C, N);
        cudaDeviceSynchronize();
        clock_t end = clock();

        double elapsed = (double)(end - start) / CLOCKS_PER_SEC;
        printf("GPU execution time (N=%d): %f seconds\n", N, elapsed);

        cudaFree(A); cudaFree(B); cudaFree(C);
    }
    cublasDestroy(handle);
    return 0;
}

Overwriting matrix_gpu_cublas.cu


In [ ]:
!nvcc -arch=sm_75 matrix_gpu_cublas.cu -o matrix_gpu_cublas_run -lcublas

In [ ]:
!./matrix_gpu_cublas_run 2048

GPU execution time (N=512): 0.009214 seconds
GPU execution time (N=1024): 0.003726 seconds
GPU execution time (N=2048): 0.016172 seconds


In [ ]:
%%writefile matrix_lib.cu
#include <cuda_runtime.h>
#include <stdio.h>

#define TILE_WIDTH 16

__global__ void matrixMultiplyTiled(float *A, float *B, float *C, int N) {
    __shared__ float ds_A[TILE_WIDTH][TILE_WIDTH];
    __shared__ float ds_B[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x; int by = blockIdx.y;
    int tx = threadIdx.x; int ty = threadIdx.y;

    int Row = by * TILE_WIDTH + ty;
    int Col = bx * TILE_WIDTH + tx;

    float Pvalue = 0.0;
    for (int m = 0; m < (N + TILE_WIDTH - 1) / TILE_WIDTH; ++m) {
        if (Row < N && (m*TILE_WIDTH+tx) < N)
            ds_A[ty][tx] = A[Row * N + m * TILE_WIDTH + tx];
        else
            ds_A[ty][tx] = 0.0f;

        if (Col < N && (m*TILE_WIDTH+ty) < N)
            ds_B[ty][tx] = B[(m*TILE_WIDTH + ty) * N + Col];
        else
            ds_B[ty][tx] = 0.0f;

        __syncthreads();

        for (int k = 0; k < TILE_WIDTH; ++k)
            Pvalue += ds_A[ty][k] * ds_B[k][tx];
        __syncthreads();
    }

    if (Row < N && Col < N)
        C[Row * N + Col] = Pvalue;
}

// Exposed C function for Python
extern "C" void gpu_matrix_multiply(float *h_A, float *h_B, float *h_C, int
N) {
    size_t size = N * N * sizeof(float);
    float *d_A, *d_B, *d_C;

    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    dim3 dimBlock(TILE_WIDTH, TILE_WIDTH);
    dim3 dimGrid((N + TILE_WIDTH - 1) / TILE_WIDTH, (N + TILE_WIDTH - 1) /
TILE_WIDTH);

    matrixMultiplyTiled<<<dimGrid, dimBlock>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize();

    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
}

Writing matrix_lib.cu


In [ ]:
!nvcc -Xcompiler -fPIC -shared matrix_lib.cu -o libmatrix.so

In [ ]:
import ctypes
import numpy as np
import time

# Load shared library
lib = ctypes.cdll.LoadLibrary("./libmatrix.so")

# Define argument types
lib.gpu_matrix_multiply.argtypes = [
np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
ctypes.c_int
]
N = 1024
A = np.random.rand(N, N).astype(np.float32)
B = np.random.rand(N, N).astype(np.float32)
C = np.zeros((N, N), dtype=np.float32)
start = time.time()
lib.gpu_matrix_multiply(A.ravel(), B.ravel(), C.ravel(), N)
end = time.time()
print(f"Python call to CUDA library completed in {end - start:.4f} seconds")

Python call to CUDA library completed in 0.1164 seconds


In [7]:
from google.colab import drive
import os

drive.mount('/content/drive')
project_path = '/content/drive/MyDrive/Colab Notebooks/scripts'
os.makedirs(project_path, exist_ok=True)
os.chdir(project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
os.path.exists("test_1.txt")

True

In [19]:
%%writefile convolution_gpu.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

__global__ void convolutionGPU(unsigned int *input, unsigned int *output, float *kernel, int M, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int offset = N / 2;

    if (row < M && col < M) {
        float sum = 0.0f;
        for (int ki = 0; ki < N; ki++) {
            for (int kj = 0; kj < N; kj++) {
                int ii = row + ki - offset;
                int jj = col + kj - offset;
                if (ii >= 0 && ii < M && jj >= 0 && jj < M) {
                    sum += input[ii * M + jj] * kernel[ki * N + kj];
                }
            }
        }
        if (sum < 0.0f) sum = 0.0f;
        if (sum > 255.0f) sum = 255.0f;
        output[row * M + col] = (unsigned int)sum;
    }
}

unsigned int* readMatrixManaged(const char *filename, int *M) {
    FILE *fp = fopen(filename, "r");
    if (!fp) return NULL;
    int cols;
    if (fscanf(fp, "%d %d", M, &cols) != 2) {
        fclose(fp);
        return NULL;
    }
    unsigned int *data;
    cudaMallocManaged(&data, *M * *M * sizeof(unsigned int));
    for (int i = 0; i < *M * *M; i++) {
        fscanf(fp, "%u", &data[i]);
    }
    fclose(fp);
    return data;
}

void saveMatrix(const char *filename, unsigned int *data, int M) {
    FILE *fp = fopen(filename, "w");
    if (!fp) return;
    fprintf(fp, "%d %d\n", M, M);
    for (int i = 0; i < M * M; i++) {
        fprintf(fp, "%u ", data[i]);
        if ((i + 1) % M == 0) fprintf(fp, "\n");
    }
    fclose(fp);
}

float* generateFilterManaged(int N) {
    float *filter;
    cudaMallocManaged(&filter, N * N * sizeof(float));
    if (N == 3) {
        float laplacian[9] = {0, -1, 0, -1, 4, -1, 0, -1, 0};
        for(int i=0; i<9; i++) filter[i] = laplacian[i];
    } else {
        float val = 1.0f / (N * N);
        for (int i = 0; i < N * N; i++) {
            filter[i] = val;
        }
    }
    return filter;
}

int main(int argc, char **argv) {
    const char *filenames[] = {"test_1.txt", "test_2.txt", "test_3.txt"};
    int filter_sizes[] = {3, 5, 7};
    int num_images = 3;
    int num_filters = 3;

    for (int i = 0; i < num_images; i++) {
        const char *input_path = filenames[i];

        int M;
        unsigned int *input = readMatrixManaged(input_path, &M);
        if (!input) {
            printf("Failed to read file: %s\n", input_path);
            continue;
        }

        unsigned int *output;
        cudaMallocManaged(&output, M * M * sizeof(unsigned int));

        for (int j = 0; j < num_filters; j++) {
            int N = filter_sizes[j];
            float *kernel = generateFilterManaged(N);

            dim3 threadsPerBlock(16, 16);
            dim3 blocksPerGrid((M + threadsPerBlock.x - 1) / threadsPerBlock.x,
                               (M + threadsPerBlock.y - 1) / threadsPerBlock.y);

            clock_t start = clock();
            convolutionGPU<<<blocksPerGrid, threadsPerBlock>>>(input, output, kernel, M, N);
            cudaDeviceSynchronize();
            clock_t end = clock();

            double elapsed = (double)(end - start) / CLOCKS_PER_SEC;
            printf("Image: %s (MxM=%dx%d), Filter: %dx%d, Time: %f s\n",
                   filenames[i], M, M, N, N, elapsed);

            char output_path[64];
            sprintf(output_path, "result_%d_%dx%d.txt", M, N, N);
            saveMatrix(output_path, output, M);

            cudaFree(kernel);
        }

        cudaFree(input);
        cudaFree(output);
    }
    return 0;
}

Overwriting convolution_gpu.cu


In [20]:
!nvcc -arch=sm_75 convolution_gpu.cu -o convolution_gpu_run

In [21]:
!./convolution_gpu_run

Image: test_1.txt (MxM=2048x2048), Filter: 3x3, Time: 0.008173 s
Image: test_1.txt (MxM=2048x2048), Filter: 5x5, Time: 0.006555 s
Image: test_1.txt (MxM=2048x2048), Filter: 7x7, Time: 0.006922 s
Image: test_2.txt (MxM=1024x1024), Filter: 3x3, Time: 0.004860 s
Image: test_2.txt (MxM=1024x1024), Filter: 5x5, Time: 0.003811 s
Image: test_2.txt (MxM=1024x1024), Filter: 7x7, Time: 0.001347 s
Image: test_3.txt (MxM=512x512), Filter: 3x3, Time: 0.000082 s
Image: test_3.txt (MxM=512x512), Filter: 5x5, Time: 0.000058 s
Image: test_3.txt (MxM=512x512), Filter: 7x7, Time: 0.000635 s


In [22]:
%%writefile convolution_lib.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void convolutionGPU(unsigned int *input, unsigned int *output, float *kernel, int M, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int offset = N / 2;

    if (row < M && col < M) {
        float sum = 0.0f;
        for (int ki = 0; ki < N; ki++) {
            for (int kj = 0; kj < N; kj++) {
                int ii = row + ki - offset;
                int jj = col + kj - offset;
                if (ii >= 0 && ii < M && jj >= 0 && jj < M) {
                    sum += input[ii * M + jj] * kernel[ki * N + kj];
                }
            }
        }
        if (sum < 0.0f) sum = 0.0f;
        if (sum > 255.0f) sum = 255.0f;
        output[row * M + col] = (unsigned int)sum;
    }
}

extern "C" void gpu_convolution_wrapper(unsigned int *h_input, unsigned int *h_output, float *h_kernel, int M, int N) {
    unsigned int *d_input, *d_output;
    float *d_kernel;
    size_t img_size = M * M * sizeof(unsigned int);
    size_t kern_size = N * N * sizeof(float);

    cudaMalloc((void**)&d_input, img_size);
    cudaMalloc((void**)&d_output, img_size);
    cudaMalloc((void**)&d_kernel, kern_size);

    cudaMemcpy(d_input, h_input, img_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_kernel, h_kernel, kern_size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGrid((M + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (M + threadsPerBlock.y - 1) / threadsPerBlock.y);

    convolutionGPU<<<blocksPerGrid, threadsPerBlock>>>(d_input, d_output, d_kernel, M, N);
    cudaDeviceSynchronize();

    cudaMemcpy(h_output, d_output, img_size, cudaMemcpyDeviceToHost);

    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_kernel);
}

Writing convolution_lib.cu


In [23]:
!nvcc -Xcompiler -fPIC -shared -o convolution_lib.so convolution_lib.cu

In [ ]:
import ctypes
import numpy as np
import os
import time

def get_kernel(N):
    if N == 3:
        return np.array([0, -1, 0, -1, 4, -1, 0, -1, 0], dtype=np.float32)
    else:
        return np.full(N * N, 1.0 / (N * N), dtype=np.float32)

lib = ctypes.CDLL('./convolution_lib.so')
lib.gpu_convolution_wrapper.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.uint32, ndim=1),
    np.ctypeslib.ndpointer(dtype=np.uint32, ndim=1),
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1),
    ctypes.c_int,
    ctypes.c_int
]

filenames = ["test_1.txt", "test_2.txt", "test_3.txt"]
filter_sizes = [3, 5, 7]

for fname in filenames:
    if not os.path.exists(fname):
        print(f"File not found: {fname}")
        continue

    with open(fname, 'r') as f:
        header = f.readline().split()
        M = int(header[0])

    h_input = np.loadtxt(fname, skiprows=1, dtype=np.uint32).flatten()
    h_output = np.zeros_like(h_input)

    for N in filter_sizes:
        h_kernel = get_kernel(N)

        start_time = time.time()
        lib.gpu_convolution_wrapper(h_input, h_output, h_kernel, M, N)
        end_time = time.time()

        print(f"Image: {fname} (MxM={M}x{M}), Filter: {N}x{N}, Time: {end_time - start_time:.6f} s")

        out_name = f"result_{M}_{N}x{N}.txt"
        np.savetxt(out_name, h_output.reshape(M, M), fmt='%u', header=f"{M} {M}", comments='')

Image: test_1.txt (MxM=2048x2048), Filter: 3x3, Time: 0.166284 s
Image: test_1.txt (MxM=2048x2048), Filter: 5x5, Time: 0.011030 s
Image: test_1.txt (MxM=2048x2048), Filter: 7x7, Time: 0.012252 s
Image: test_2.txt (MxM=1024x1024), Filter: 3x3, Time: 0.003047 s
Image: test_2.txt (MxM=1024x1024), Filter: 5x5, Time: 0.004043 s
Image: test_2.txt (MxM=1024x1024), Filter: 7x7, Time: 0.004118 s
Image: test_3.txt (MxM=512x512), Filter: 3x3, Time: 0.001529 s
Image: test_3.txt (MxM=512x512), Filter: 5x5, Time: 0.001608 s
Image: test_3.txt (MxM=512x512), Filter: 7x7, Time: 0.001569 s
